In [1]:
from dotenv import load_dotenv
from openai import AsyncOpenAI
from agents import Agent, Runner, trace, function_tool, OpenAIChatCompletionsModel, input_guardrail, GuardrailFunctionOutput
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
from pydantic import BaseModel

In [2]:
load_dotenv(override=True)

True

In [3]:
import logfire
logfire.configure(
    service_name="sale email manager",
    send_to_logfire=False
)
logfire.instrument_openai_agents()

In [4]:
from langfuse import get_client
langfuse = get_client()
langfuse.auth_check()

True

In [5]:
openai_api_key = os.getenv('OPENAI_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
openrouter_api_key = os.getenv('OR_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')


In [6]:
instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

In [7]:
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
OPENROUTER_BASE_URL = "https://openrouter.ai/api/v1"
GROQ_BASE_URL = "https://api.groq.com/openai/v1"

In [8]:

deepseek_client = AsyncOpenAI(base_url=OPENROUTER_BASE_URL, api_key=openrouter_api_key)
gemini_client = AsyncOpenAI(base_url=GEMINI_BASE_URL, api_key=google_api_key)
groq_client = AsyncOpenAI(base_url=GROQ_BASE_URL, api_key=groq_api_key)

deepseek_model = OpenAIChatCompletionsModel(model="deepseek/deepseek-chat-v3.1:free", openai_client=deepseek_client)
gemini_model = OpenAIChatCompletionsModel(model="gemini-2.0-flash", openai_client=gemini_client)
llama3_3_model = OpenAIChatCompletionsModel(model="llama-3.3-70b-versatile", openai_client=groq_client)

In [9]:
sales_agent1 = Agent(name="DeepSeek Sales Agent", instructions=instructions1, model=deepseek_model)
sales_agent2 =  Agent(name="Gemini Sales Agent", instructions=instructions2, model=gemini_model)
sales_agent3  = Agent(name="Llama3.3 Sales Agent",instructions=instructions3,model=llama3_3_model)
res = await Runner.run(sales_agent3, "Write a cold email to a CTO of a mid-size startup about ComplAI")
res.final_output

17:35:41.600 OpenAI Agents trace: Agent workflow
17:35:41.604   Agent run: 'Llama3.3 Sales Agent'
17:35:41.606     Chat completion with 'llama-3.3-70b-versatile' [LLM]


"Subject: Simplify SOC2 Compliance with ComplAI\n\nDear [CTO's Name],\n\nAs a CTO of a growing startup, ensuring SOC2 compliance is crucial for building trust with your customers and stakeholders. However, the audit process can be time-consuming and cumbersome.\n\nComplAI is here to help. Our AI-powered SaaS tool streamlines SOC2 compliance and audit preparation, saving you time and resources. With ComplAI, you can:\n\n* Automate evidence collection and mapping\n* Identify and remediate gaps in your compliance posture\n* Stay on top of changing regulatory requirements\n\nOur platform has already helped numerous mid-size startups like yours achieve SOC2 compliance efficiently. I'd love to schedule a quick call to discuss how ComplAI can support your compliance journey.\n\nAre you available for a 15-minute call this week or next?\n\nBest,\n[Your Name]\nSales Agent, ComplAI\n[Your Email]\n[Your Phone Number]"

In [10]:
description = "Write a cold sales email"

tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description=description)
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description=description)
tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description=description)

In [11]:
sender_email = "bhavin@advant.xyz"
receiver_email = 'bhavin2004k@gmail.com'

In [12]:
@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email(sender_email)
    to_email = To(receiver_email) 
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    response = sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}
send_html_email

FunctionTool(name='send_html_email', description='Send out an email with the given subject and HTML body to all sales prospects', params_json_schema={'properties': {'subject': {'title': 'Subject', 'type': 'string'}, 'html_body': {'title': 'Html Body', 'type': 'string'}}, 'required': ['subject', 'html_body'], 'title': 'send_html_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x7f1e0427e5c0>, strict_json_schema=True, is_enabled=True)

In [13]:
from pydantic import BaseModel

class EmailRequest(BaseModel):
    subject: str
    html_body: str

In [14]:

instructions_to_mail = '''Write a professional, concise email subject line and body based on the following cold sales email.
Make sure the email is engaging and likely to get a response.
You will write a professional subject line for the cold sales email.
And The body should be in HTML format which is also beautiful and engaging make user feel into love with the email.
You will be given the body text of the email so you don't need to come up with the body text.
Just format it in HTML.

Return the subject line and body in the json format:
{
  "subject": "subject line here",
  "html_body": "email body here"
}
'''

email_writer_agent = Agent(instructions=instructions_to_mail,model="gpt-4o-mini",name="Email Writer Agent",output_type=EmailRequest)
res = await Runner.run(email_writer_agent, "Write a cold sales email to a CTO of a mid-size startup about ComplAI")
email_writer_agent_tool = email_writer_agent.as_tool(tool_name="email_writer_agent", tool_description="Write a professional, concise email subject line and body based on the following cold sales email.And respond in json format.")

17:35:43.147 OpenAI Agents trace: Agent workflow
17:35:43.150   Agent run: 'Email Writer Agent'
17:35:43.178     Responses API with 'gpt-4o-mini'


In [15]:
res.final_output

EmailRequest(subject='Revolutionize Your AI Strategy with ComplAI', html_body="<html>\n<head>\n<style>\nbody {font-family: Arial, sans-serif; color: #333; line-height: 1.6;}\nh1 {color: #007bff;}\np {margin: 0 0 15px;}\na {color: #007bff; text-decoration: none;}\n.container {max-width: 600px; margin: 0 auto; padding: 20px; border: 1px solid #e0e0e0; border-radius: 8px;}\n.button {background-color: #007bff; color: #fff; padding: 10px 15px; text-decoration: none; border-radius: 5px;}\n</style>\n</head>\n<body>\n<div class='container'>\n    <h1>Hello [CTO's Name],</h1>\n    <p>I hope this message finds you well. I'm reaching out to share how ComplAI can transform your AI initiatives into impactful solutions.</p>\n    <p>At ComplAI, we specialize in streamlining AI integration for startups like yours, enabling you to focus on innovation while we handle the complexities of AI deployment.</p>\n    <p>Whether it’s enhancing user experience, automating processes, or driving data-driven decisio

In [16]:
email_tools = [email_writer_agent_tool, send_html_email]

In [17]:
instructions ="You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the email_writer_agent tool to write a subject for the email and convert body text to HTML. \
Finally, you use the send_html_email tool to send the email with the subject and HTML body."


emailer_agent = Agent(
    name="Email Manager",
    instructions=instructions,
    tools=email_tools,
    model="gpt-4o-mini",
    handoff_description="Convert an email to the subject and html body and then send it mail")

In [18]:
tools = [tool1, tool2, tool3]
handoffs = [emailer_agent]

In [20]:
sales_manager_instructions = """
You are a Sales Manager at ComplAI. Your goal is to find the single best cold sales email using the sales_agent tools.
 
Follow these steps carefully:
1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.
 
2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.
You can use the tools multiple times if you're not satisfied with the results from the first try.
 
3. Handoff for Sending: Pass ONLY the winning email draft to the 'Email Manager' agent. The Email Manager will take care of formatting and sending.
 
Crucial Rules:
- You must use the sales agent tools to generate the drafts — do not write them yourself.
- You must hand off exactly ONE email to the Email Manager — never more than one.
"""


sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=handoffs,
    model="gpt-4o-mini")

message = "Send out a cold sales email addressed to Dear CEO from Alice"

with trace("Automated SDR"):
    result = await Runner.run(sales_manager, message)

17:35:59.061 OpenAI Agents trace: Automated SDR
17:35:59.062   Agent run: 'Sales Manager'
17:35:59.064     Responses API with 'gpt-4o-mini'
17:36:10.548     Function: sales_agent1
17:36:10.550     Function: sales_agent2
17:36:10.551     Function: sales_agent3
                 Function: sales_agent1
17:36:10.552       Agent run: 'DeepSeek Sales Agent'
                 Function: sales_agent2
17:36:10.553       Agent run: 'Gemini Sales Agent'
                 Function: sales_agent3
17:36:10.554       Agent run: 'Llama3.3 Sales Agent'
                 Function: sales_agent1
                   Agent run: 'DeepSeek Sales Agent'
17:36:10.555         Chat completion with 'deepseek/deepseek-chat-v3.1:free' [LLM]
                 Function: sales_agent2
                   Agent run: 'Gemini Sales Agent'
17:36:10.557         Chat completion with 'gemini-2.0-flash' [LLM]
                 Function: sales_agent3
                   Agent run: 'Llama3.3 Sales Agent'
17:36:10.559         Chat completion

Context error: No active span in current context. Operations that depend on an active span will be skipped. Ensure spans are created with start_as_current_span() or that you're operating within an active span context.


In [21]:
result.final_output

'The email has been successfully sent to the CEO with the following details:\n\n### Subject:\n**Unlock New Opportunities with ComplAI**\n\n### Body:\n```html\n<html>\n<head>\n    <style>\n        body { font-family: Arial, sans-serif; background-color: #f4f4f4; padding: 20px; }\n        .container { background-color: #ffffff; border-radius: 8px; box-shadow: 0 2px 4px rgba(0, 0, 0, 0.1); padding: 20px; }\n        h1 { color: #333; }\n        p { color: #666; line-height: 1.6; }\n        .button { background-color: #007BFF; color: #ffffff; padding: 10px 15px; text-decoration: none; border-radius: 5px; }\n        .footer { margin-top: 20px; font-size: 0.8em; color: #999; }\n    </style>\n</head>\n<body>\n    <div class="container">\n        <h1>Dear CEO,</h1>\n        <p>I hope this message finds you well! I\'m Alice, reaching out from ComplAI to introduce you to a powerful solution that can drive efficiency and innovation in your organization.</p>\n        <p>At ComplAI, we specialize in

## Check out the trace:

https://platform.openai.com/traces

In [26]:
class NameCheckOutput(BaseModel):
    is_name_in_message: bool
    name: str

guardrail_agent = Agent( 
    name="Name check",
    instructions="Check if the user is including someone's personal name in what they want you to do.",
    output_type=NameCheckOutput,
    model="gpt-4o-mini"
)

In [27]:
@input_guardrail
async def guardrail_against_name(ctx, agent, message):
    result = await Runner.run(guardrail_agent, message, context=ctx.context)
    is_name_in_message = result.final_output.is_name_in_message
    return GuardrailFunctionOutput(output_info={"found_name": result.final_output},tripwire_triggered=is_name_in_message)

In [28]:
careful_sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=[emailer_agent],
    model="gpt-4o-mini",
    input_guardrails=[guardrail_against_name]
    )

message = "Send out a cold sales email addressed to Dear CEO from Alice"

with trace("Protected Automated SDR"):
    result = await Runner.run(careful_sales_manager, message)

17:41:00.420 OpenAI Agents trace: Protected Automated SDR
17:41:00.421   Agent run: 'Sales Manager'
17:41:00.422     Guardrail 'guardrail_against_name' triggered=False
17:41:00.424       Agent run: 'Name check'
17:41:00.425     Responses API with 'gpt-4o-mini'
                 Guardrail 'guardrail_against_name' triggered=False
                   Agent run: 'Name check'
17:41:00.428         Responses API with 'gpt-4o-mini'


InputGuardrailTripwireTriggered: Guardrail InputGuardrail triggered tripwire

17:41:12.677 Function: sales_agent1
17:41:12.677 Function: sales_agent2
17:41:12.678 Function: sales_agent3
             Function: sales_agent1
17:41:12.678   Agent run: 'DeepSeek Sales Agent'
             Function: sales_agent2
17:41:12.678   Agent run: 'Gemini Sales Agent'
             Function: sales_agent3
17:41:12.679   Agent run: 'Llama3.3 Sales Agent'
             Function: sales_agent1
               Agent run: 'DeepSeek Sales Agent'
17:41:12.679     Chat completion with 'deepseek/deepseek-chat-v3.1:free' [LLM]
             Function: sales_agent2
               Agent run: 'Gemini Sales Agent'
17:41:12.681     Chat completion with 'gemini-2.0-flash' [LLM]
             Function: sales_agent3
               Agent run: 'Llama3.3 Sales Agent'
17:41:12.683     Chat completion with 'llama-3.3-70b-versatile' [LLM]


## Check out the trace:

https://platform.openai.com/traces

In [29]:

message = "Send out a cold sales email addressed to Dear CEO from Head of Business Development"

with trace("Protected Automated SDR"):
    result = await Runner.run(careful_sales_manager, message)

17:49:06.033 OpenAI Agents trace: Protected Automated SDR
17:49:06.035   Agent run: 'Sales Manager'
17:49:06.036     Guardrail 'guardrail_against_name' triggered=False
17:49:06.037       Agent run: 'Name check'
17:49:06.039     Responses API with 'gpt-4o-mini'
                 Guardrail 'guardrail_against_name' triggered=False
                   Agent run: 'Name check'
17:49:06.043         Responses API with 'gpt-4o-mini'
17:49:14.601     Function: sales_agent1
17:49:14.601     Function: sales_agent2
17:49:14.602     Function: sales_agent3
                 Function: sales_agent1
17:49:14.602       Agent run: 'DeepSeek Sales Agent'
                 Function: sales_agent2
17:49:14.604       Agent run: 'Gemini Sales Agent'
                 Function: sales_agent3
17:49:14.605       Agent run: 'Llama3.3 Sales Agent'
                 Function: sales_agent1
                   Agent run: 'DeepSeek Sales Agent'
17:49:14.606         Chat completion with 'deepseek/deepseek-chat-v3.1:free' [LLM]
 